In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Test set', (10000, 28, 28), (10000,))
('Validation set', (10000, 28, 28), (10000,))


In [3]:
image_height = 28
image_width = 28
num_labels = 10

train_dataset = train_dataset.reshape((-1, image_height * image_width)).astype(np.float32)
test_dataset = test_dataset.reshape((-1, image_height * image_width)).astype(np.float32)
valid_dataset = valid_dataset.reshape((-1, image_height * image_width)).astype(np.float32)

print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)

('Training set', (200000, 784), (200000,))
('Test set', (10000, 784), (10000,))
('Validation set', (10000, 784), (10000,))


In [4]:
def plot_image(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = 'Greys', interpolation="nearest")
    plt.axis("off")
    pass

In [5]:
plot_image(train_dataset[3])

In [6]:
train_labels[3]

6

# l2 regularization

In [7]:
batch_size = 50
epochs = 101
beta = 0.01

tf.reset_default_graph()

with tf.name_scope('dnn'):
    tf_dataset = tf.placeholder(tf.float32, shape = [None, image_width * image_height])
    tf_labels = tf.placeholder(tf.int32, shape = (None))
    
    weights = tf.Variable(tf.truncated_normal([image_width * image_width, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.add(tf.matmul(tf_dataset, weights), biases)
    pass

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels=tf_labels)
    loss = tf.reduce_mean(xentropy)
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + regularizer * beta)
    loss_summary = tf.summary.scalar('log_loss', loss)
    pass
    
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    predicted = tf.nn.in_top_k(logits, tf_labels, 1)
    accuracy = tf.reduce_mean(tf.cast(predicted, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    pass

In [8]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter('/tmp/notmnist/notmnist_dnn-l2regularization', tf.get_default_graph())

# Train the model

In [9]:
with tf.Session() as sess:
    init.run()

    for epoch in range(epochs):
        for iteration in range(train_dataset.shape[0] // epochs):
            batch_index = np.random.choice(train_labels.shape[0], batch_size)
            batch_data, batch_labels = train_dataset[batch_index], train_labels[batch_index]
            feed_dict = {tf_dataset: batch_data, tf_labels: batch_labels}
            _, accuracy_val, loss_val = sess.run([training_op, accuracy, loss], feed_dict=feed_dict)
        
        test_acc_val = sess.run(accuracy, feed_dict = {tf_dataset: test_dataset, tf_labels: test_labels})
        valid_acc_val, valid_loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], 
                        feed_dict = {tf_dataset: valid_dataset, tf_labels: valid_labels})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        
        
        if epoch % 5 == 0:
            print ('Mini Batch Iteration %d, Train Loss: %.3f, Train Accuracy: %.3f' % (epoch, loss_val, accuracy_val))
            print ('Test Accuracy: %.3f, Validation Accuracy: %.3f' % (test_acc_val, valid_acc_val))
            print ('\n')
            
    save_path = saver.save(sess, "./model4.ckpt")
    pass

Mini Batch Iteration 0, Train Loss: 21.777, Train Accuracy: 0.660
Test Accuracy: 0.714, Validation Accuracy: 0.646


Mini Batch Iteration 5, Train Loss: 3.167, Train Accuracy: 0.860
Test Accuracy: 0.862, Validation Accuracy: 0.786


Mini Batch Iteration 10, Train Loss: 1.170, Train Accuracy: 0.780
Test Accuracy: 0.892, Validation Accuracy: 0.822


Mini Batch Iteration 15, Train Loss: 0.603, Train Accuracy: 0.840
Test Accuracy: 0.894, Validation Accuracy: 0.826


Mini Batch Iteration 20, Train Loss: 0.803, Train Accuracy: 0.800
Test Accuracy: 0.894, Validation Accuracy: 0.829


Mini Batch Iteration 25, Train Loss: 0.381, Train Accuracy: 0.920
Test Accuracy: 0.895, Validation Accuracy: 0.830


Mini Batch Iteration 30, Train Loss: 0.449, Train Accuracy: 0.880
Test Accuracy: 0.893, Validation Accuracy: 0.828


Mini Batch Iteration 35, Train Loss: 0.542, Train Accuracy: 0.880
Test Accuracy: 0.893, Validation Accuracy: 0.828


Mini Batch Iteration 40, Train Loss: 0.587, Train Accuracy: 0.900